In [4]:
using StaticArrays
using LinearAlgebra
using Combinatorics
sa = StaticArrays
la = LinearAlgebra
cm = Combinatorics 

Combinatorics

In [5]:
SPoint = sa.SVector{2, Float64}
SamePoints = Tuple{SPoint, SPoint, SPoint, SPoint}
function first_in_klein(point::SPoint)::SPoint
    x::Float64 = point[1]
    y::Float64 = point[2]
    if (0 <= x <= 1) & (0 <= y <= 1)
        return point

    elseif -1 <= x < 0 < y <= 1
        return @sa.SVector [x + 2.0, 2.0 - y]

    elseif (-1 <= x < 0) & (-1 <= y <= 0)
        return @sa.SVector [x + 2.0, -y]

    elseif (-1 <= y < 0 <= x <=1)
        return @sa.SVector [x, 2.0 + y]
    end
    throw(DomainError(point, "Make sure that the first and second coordinates are in the intreval [-1, 1]"))
end

function samepoints_in_kelin(point:: SPoint)::SamePoints
    firstpoint:: SPoint = first_in_klein(point)
    x:: Float64 = firstpoint[1]
    y:: Float64 = firstpoint[2]
    second:: SPoint = @sa.SVector [x - 2.0, 2.0 - y]
    theard:: SPoint = @sa.SVector [x - 2.0, -y]
    fourth:: SPoint =  @sa.SVector [x, y - 2.0]
    return (firstpoint, second, theard, fourth)
end


samepoints_in_kelin (generic function with 1 method)

In [6]:
point = @sa.SVector [0.5, -1.0]
println(samepoints_in_kelin(point))

([0.5, 1.0], [-1.5, 1.0], [-1.5, -1.0], [0.5, -1.0])


In [7]:
Quadrants = Tuple{Int64, Int64}

function quadrants_square_distance(quadrants:: Quadrants, samepoints1:: SamePoints, samepoints2:: SamePoints)::Float64
    index1:: Int64 = quadrants[1]
    index2:: Int64 = quadrants[2]
    diff:: SPoint = samepoints1[index1]- samepoints2[index2]
    return la.norm(diff)
end

function candidates_quadrants(point1:: SPoint, point2:: SPoint)::Tuple{Quadrants, Quadrants}
    add_second:: Float64 = (point1 + point2)[2]
    diff:: SPoint = point1 - point2
    #print(diff[2], 'in candidate_quadrants')
    #candidate1:: Quadrants = (1, 3) if (add_second < 1.0)  else (1, 2) if (add_second < 3.0) else (4, 2)
    candidate1:: Quadrants = add_second < 1.0 ? (1, 3) : add_second < 3.0 ? (1, 2) : (4, 2)
    candidate2:: Quadrants =  diff[2] < -1.0 ? (1, 4) : diff[2] < 1.0 ? (1, 1) : (2, 3)
    return (diff[1] >= 0 ? reverse(candidate1) : candidate1, candidate2)
end


candidates_quadrants (generic function with 1 method)

In [65]:

function distance_in_klein_bottle(point1:: SPoint, point2:: SPoint)::Float64
    samepoints1:: SamePoints = samepoints_in_kelin(point1)
    samepoints2:: SamePoints = samepoints_in_kelin(point2)
    if samepoints1 === samepoints2
        return 0.0
    end
    point1_in_first:: SPoint = samepoints1[1]
    point2_in_first:: SPoint = samepoints2[1]
    candidates:: Tuple{Quadrants, Quadrants} = candidates_quadrants(point1_in_first, point2_in_first)
    #print(candidates, 'in dist klein')
    dist1:: Float64 = quadrants_square_distance(candidates[1], samepoints1, samepoints2)
    dist2:: Float64 = quadrants_square_distance(candidates[2], samepoints1, samepoints2)
    #print(dist1, dist2)
    distance:: Float64 = minimum((dist1, dist2))
end

function matrix_index(;matrix_size::Tuple{Int64, Int64}, index::Int64):: Tuple{Int64, Int64}
    row:: Int64 = matrix_size[1]
    column:: Int64 = matrix_size[2]
    remainder:: Int64 = rem(index, row)
    division:: Int64 = div(index, row)
    if index > row * column
        throw(DomainError(index, "Make sure that index <= row * column"))
    end

    return remainder !== 0 ? (remainder, division + 1) : (row, division)
    
end

function matrix_index(indexies:: Tuple{Int64, Int64};matrix_size::Tuple{Int64, Int64}):: Int64
    i:: Int64, j:: Int64 = indexies
    row:: Int64, column:: Int64 = matrix_size
    index:: Int64 = (j - 1) * row + i
    if (i > row) || (j > column)
        throw(DomainError(index, "Make sure that each index <= row , column"))
    end
    return index
end

function distance_matrix(pointcloud:: Vector{SPoint}):: Array{Float64, 2}
    point_num:: Int64 = size(pointcloud, 1)
    indexcombs:: Base.Generator{Combinatorics.Combinations,Combinatorics.var"#reorder#10"{UnitRange{Int64}}} = cm.combinations(1:point_num, 2)
    dist_matrix:: Array{Float64, 2} = zeros(point_num, point_num)
    for indexies = indexcombs
        i::Int64, j:: Int64 = reverse(Tuple(indexies))
        dist_matrix[matrix_index((i, j), matrix_size=(point_num, point_num))] = distance_in_klein_bottle(pointcloud[i], pointcloud[j])
    end
    #return map(points:: Vector{SPoint}-> distance_in_klein_bottle(points[1], points[2])::Float64, combs)
    return dist_matrix + transpose(dist_matrix)
end



distance_matrix (generic function with 1 method)

In [52]:
index = matrix_index((2, 1), matrix_size=(5, 5))
println(index)
matrix_index(matrix_size=(5, 5), index=index)

2


(2, 1)

In [10]:
point1 = @sa.SVector [1.0, 0.3]
point2 = @sa.SVector [-1.0, -0.3]
println(distance_in_klein_bottle(point1, point2))

0.0


In [11]:
function uniformpoints_in_square(;range_min:: Float64, range_max:: Float64, sampling:: Int64, step:: Float64 = 0.00001)::Array{Float64,}
    rng = range_min:step:range_max
    dim:: Int64 = 2
    return rand(rng, (sampling, dim))
end

function staticalize(ndarray:: Array{Float64, })::Vector{SPoint}
    rownum ::Int = size(ndarray, 1)
    return [SPoint(ndarray[i,:]) for i = 1:rownum]
end

staticalize (generic function with 1 method)

In [70]:
pointcloud = uniformpoints_in_square(range_min=-1.0, range_max=1.0, sampling=1000)
staticpointcloud = staticalize(pointcloud)


1000-element Array{SArray{Tuple{2},Float64,1,2},1}:
 [0.45933, 0.25494]
 [-0.92605, 0.60225]
 [-0.20641, -0.39875]
 [0.53317, 0.87389]
 [-0.58743, 0.00425]
 [-0.56882, -0.59475]
 [0.77383, -0.49757]
 [0.97408, -0.42162]
 [-0.27587, 0.10736]
 [-0.16544, -0.49806]
 [-0.68427, 0.71138]
 [-0.94733, -0.15295]
 [-0.58827, 0.67882]
 ⋮
 [0.26336, -0.54316]
 [0.16633, 0.02727]
 [0.59557, -0.93973]
 [-0.86636, -0.54063]
 [0.6356, -0.91488]
 [-0.66095, -0.17736]
 [-0.76755, -0.29667]
 [-0.24801, -0.17966]
 [0.03986, 0.02627]
 [0.27096, 0.38526]
 [0.07407, 0.2178]
 [0.54841, -0.4713]

In [68]:

println(staticpointcloud[1])
println(staticpointcloud[2])
println(distance_in_klein_bottle(staticpointcloud[1], staticpointcloud[2]) )
println(distance_in_klein_bottle(staticpointcloud[2], staticpointcloud[1]))


[-0.54727, 0.95327]
[-0.87687, -0.83779]
0.39024618332534655
0.39024618332534655


In [71]:
@time distance_matrix(staticpointcloud)


  1.652555 seconds (18.14 M allocations: 600.819 MiB, 6.13% gc time)


1000×1000 Array{Float64,2}:
 0.0       1.05477   0.933017  0.623339  …  0.229056  0.387046  0.731683
 1.05477   0.0       1.23121   0.752909     1.21652   1.07147   0.541609
 0.933017  1.23121   0.0       1.03732      0.917907  0.67735   0.758299
 0.623339  0.752909  1.03732   0.0          0.554539  0.800766  0.654987
 0.987849  0.687217  0.554604  1.24277      0.93915   0.695116  0.982297
 1.02955   0.878876  0.412016  0.940394  …  1.17898   1.03612   1.12403
 0.815587  0.317852  0.985209  0.673038     1.01601   1.00071   0.226946
 0.850118  0.206401  1.15957   0.831088     1.07025   1.10403   0.428559
 0.749866  0.817099  0.510854  1.1145       0.613393  0.366954  1.00712
 0.978441  1.17812   0.107429  0.939416     0.985241  0.754865  0.714351
 1.23132   0.265268  1.01006   0.885664  …  1.00937   0.904821  0.804001
 0.602042  0.7555    0.780628  0.888614     0.815499  0.980746  0.802475
 1.13011   0.34635   0.998346  0.985868     0.907994  0.806991  0.887911
 ⋮                       

In [9]:
pointcloud = uniformpoints_in_square(range_min=-1.0, range_max=1.0, sampling=1000)
staticpointcloud = staticalize(pointcloud)
staticpointcloud[1]
println(typeof(staticpointcloud))


Array{SArray{Tuple{2},Float64,1,2},1}


In [12]:
@time distance_map(staticpointcloud)

1.409229069385102